In [7]:
pip install boto3 pydantic transformers


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import boto3
from pydantic import BaseModel, Field, ValidationError
from typing import List
import json
import logging

# Set up logging for error tracking
logging.basicConfig(level=logging.INFO)

# Set up AWS Bedrock client
bedrock = boto3.client(service_name='bedrock', region_name='us-east-1')

UnknownServiceError: Unknown service: 'bedrock-runtime'. Valid service names are: accessanalyzer, account, acm, acm-pca, alexaforbusiness, amp, amplify, amplifybackend, amplifyuibuilder, apigateway, apigatewaymanagementapi, apigatewayv2, appconfig, appconfigdata, appfabric, appflow, appintegrations, application-autoscaling, application-insights, applicationcostprofiler, appmesh, apprunner, appstream, appsync, arc-zonal-shift, athena, auditmanager, autoscaling, autoscaling-plans, backup, backup-gateway, backupstorage, batch, billingconductor, braket, budgets, ce, chime, chime-sdk-identity, chime-sdk-media-pipelines, chime-sdk-meetings, chime-sdk-messaging, chime-sdk-voice, cleanrooms, cloud9, cloudcontrol, clouddirectory, cloudformation, cloudfront, cloudhsm, cloudhsmv2, cloudsearch, cloudsearchdomain, cloudtrail, cloudtrail-data, cloudwatch, codeartifact, codebuild, codecatalyst, codecommit, codedeploy, codeguru-reviewer, codeguru-security, codeguruprofiler, codepipeline, codestar, cod

In [ ]:
class Feedback(BaseModel):
    text: str
    category: str = Field(..., regex="^(Product|Shipping|Customer Service|Website)$")

class FeedbackBatch(BaseModel):
    items: List[Feedback]

In [ ]:
sample_data = [
    {"text": "The product quality is excellent, but it arrived late.", "category": "Product"},
    {"text": "I can't find the return policy on your website.", "category": "Website"},
    {"text": "Your customer service rep was very helpful.", "category": "Customer Service"},
    {"text": "The packaging was damaged when it arrived.", "category": "Shipping"}
]

try:
    feedback_batch = FeedbackBatch(items=[Feedback(**item) for item in sample_data])
except ValidationError as e:
    logging.error(f"Validation Error: {e.json()}")
    feedback_batch = None

In [ ]:
def classify_feedback(text):
    prompt = f"""Human: Classify the following customer feedback into one of these categories: Product, Shipping, Customer Service, or Website.
    Customer Feedback: "{text}"
    Classification:
    Assistant:"""
    
    try:
        # Call the Bedrock API with the correct format
        response = bedrock.invoke_model(
            modelId="anthropic.claude-v2",
            body=json.dumps({
                "prompt": prompt,
                "max_tokens_to_sample": 50,  # Adjust this based on the length of your expected output
                "temperature": 0.5,  # Adjust this parameter for controlling randomness
                "stop_sequences": ["\n"]  # Stops at the first new line after the answer
            })
        )
        # Extract and return the classification from the response
        return json.loads(response['body'].read())['completion'].strip()
    except boto3.exceptions.Boto3Error as api_error:
        logging.error(f"API call to Bedrock failed: {api_error}")
        return "Error"
    except KeyError as key_error:
        logging.error(f"Unexpected response format: {key_error}")
        return "Error"

In [ ]:
for item in feedback_batch.items:
    predicted_category = classify_feedback(item.text)
    print(f"Text: {item.text}")
    print(f"Actual Category: {item.category}")
    print(f"Predicted Category: {predicted_category}")
    print("---")